In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split as ttsplit
from sklearn.metrics import mean_squared_error as mse

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

from IPython.display import display, HTML

import eli5

from pyastrotrader import calculate_chart, calculate_aspects, calculate_transits
from pyastrotrader.utils import create_input_json
from pyastrotrader.constants import *

from settings import *

In [ ]:
SWING_TRADE_DURATION = 5
SWING_EXPECTED_VOLATILITY = 3.5
STAGNATION_THRESHOLD = 5

if 'ASSET_TO_CALCULATE' not in os.environ:
    raise ValueError("ASSET_TO_CALCULATE was not set...")
    
ASSET_TO_CALCULATE = os.environ['ASSET_TO_CALCULATE']    
NATAL_DATE = NATAL_DATES[ASSET_TO_CALCULATE]
DEFAULT_PARAMETERS = './config/default_parameters.json'
DEFAULT_CONFIG = './config/default_config.json'

SOURCE_FILE = "./input/{}_Daily".format(ASSET_TO_CALCULATE)

ETA = 0.3
DEPTH = 9
NUM_TREES = 100
MAX_INTERACTIONS = 100

param = {}
param['booster'] = 'gbtree'
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'auc'
param['tree_method'] = 'auto'
param['silent'] = 0
param['subsample'] = 0.5

PLANETS_TO_CALCULATE = [SUN,MOON,SATURN, JUPITER, VENUS, MARS]
ASPECTS_TO_CALCULATE = [CONJUNCTION, SQUARE, TRINE, OPPOSITION]


In [ ]:
StockPrices = pd.read_csv("{}.csv".format(SOURCE_FILE))
StockPrices['Counter'] = np.arange(len(StockPrices))

In [ ]:
def correct_date(x):
    date_str = str(x['Date'])
    return date_str[0:4] + '-' + date_str[4:6] + '-' + date_str[6:8]    

def change_sign(x,y):
    return  not ((x > 0 and y > 0) or (x < 0 and y < 0))

In [ ]:
def get_previous_stock_price(df, x, swing_trade_duration):
    if x['Counter'] > (swing_trade_duration - 1):
        return float(df[df['Counter'] == (x['Counter'] - swing_trade_duration)]['Price'])
    else:
        return 0

def get_future_stock_price(df, x, max, swing_trade_duration):
    if x['Counter'] < (max - swing_trade_duration):
        return float(df[df['Counter'] == (x['Counter'] + swing_trade_duration)]['Price'])
    else:
        return 0    
    
def get_future_stock_max_price(df, x, max, swing_trade_duration):
    if x['Counter'] < (max):
        current_range = df[df['Counter'] <= (x['Counter'] + swing_trade_duration)]
        current_range = current_range[current_range['Counter'] > x['Counter']]
        return current_range['High'].max()
    else:
        return 0        

def get_future_stock_min_price(df, x, max, swing_trade_duration):
    if x['Counter'] < (max):
        current_range = df[df['Counter'] <= (x['Counter'] + swing_trade_duration)]
        current_range = current_range[current_range['Counter'] > x['Counter']]
        return current_range['High'].min()
    else:
        return 0        
    
def calculate_current_trend(x):
    if x['PreviousStartPrice'] > 0:
        return ((float(x['Price']) / float(x['PreviousStartPrice'])) - 1) * 100
    else:
        return 0;

def get_previous_stock_date(df, x, swing_trade_duration):
    if x['Counter'] > (swing_trade_duration - 1):
        return float(df[df['Counter'] == (x['Counter'] - swing_trade_duration)]['Date'])
    else:
        return 0

def get_future_stock_date(df, x, max, swing_trade_duration):
    if x['Counter'] < (max - swing_trade_duration):
        return float(df[df['Counter'] == (x['Counter'] + swing_trade_duration)]['Date'])
    else:
        return 0    

def calculate_future_trend(x):
    if x['FutureFinalPrice'] > 0:
        return ((float(x['FutureFinalPrice']) / float(x['Price'])) - 1) * 100
    else:
        return 0;

def calculate_intraday_volatility(df, x, swing_trade_duration):
    current_range = df[df['Counter'] >= (x['Counter'] - (swing_trade_duration * 2))]
    current_range = current_range[current_range['Counter'] < x['Counter']]
    return (((current_range['High'] / current_range['Low']) - 1).sum() / (swing_trade_duration * 2)) * 100

In [ ]:
def calculate_swing_strenght(x):
    if float(x['CurrentTrend']) != 0 and float(x['FutureTrend'] != 0):
        return abs(float(x['CurrentTrend']) - float(x['FutureTrend']))
    else:
        return 0

def detect_swing_trade(x, swing_expected_volatility):
    return 1 if change_sign(x['FutureTrend'], x['CurrentTrend']) and \
                x['SwingStrength'] > (x['IntradayVolatility'] * swing_expected_volatility) and \
                abs(x['FutureTrend']) >= abs(x['CurrentTrend']) else 0

def clean_swing_trade(df, x, swing_trade_duration):
    if x['IsSwing'] == 1:
        current_range = df[df['Counter'] <= (x['Counter'] + swing_trade_duration)]
        current_range = current_range[current_range['Counter'] > x['Counter']]
        if current_range['IsSwing'].sum() > 0:
            return 0
        else:
            return 1
    else:
        return 0
    
def detect_price_increase(x, stagnation_threshold):
    return 1 if x['FutureTrendMax'] >= (x['Open'] * (1 + (stagnation_threshold / 100))) else 0

def clean_price_increase(df, x, swing_trade_duration):
    if x['StockIncreasedPrice'] == 1:
        current_range = df[df['Counter'] >= (x['Counter'] - swing_trade_duration)]
        current_range = current_range[current_range['Counter'] < x['Counter']]
        if current_range['StockIncreasedPrice'].sum() > 0:
            return 0
        else:
            return 1
    else:
        return 0

def detect_price_decrease(x, stagnation_threshold):
    return 1 if x['FutureTrendMin'] <= (x['Open'] * (1 - (stagnation_threshold / 100))) else 0

def clean_price_decrease(df, x, swing_trade_duration):
    if x['StockDecreasedPrice'] == 1:
        current_range = df[df['Counter'] >= (x['Counter'] - swing_trade_duration)]
        current_range = current_range[current_range['Counter'] < x['Counter']]
        if current_range['StockDecreasedPrice'].sum() > 0:
            return 0
        else:
            return 1
    else:
        return 0

def detect_price_stagnated(x, stagnation_threshold):
    return 0 if detect_price_increase(x, stagnation_threshold) or detect_price_decrease(x, stagnation_threshold) else 1

def clean_price_stagnated(df, x, swing_trade_duration):
    if x['StockStagnated'] == 1:
        current_range = df[df['Counter'] <= (x['Counter'] + swing_trade_duration)]
        current_range = current_range[current_range['Counter'] > x['Counter']]
        if current_range['StockStagnated'].sum() > 0:
            return 0
        else:
            return 1
    else:
        return 0


In [ ]:
def is_aspected(row, first_planet, second_planet, aspect):
    c_transits = transits[row['CorrectedDate']]
    found_params = [x for x in c_transits if (x['c_planet'] == first_planet and x['n_planet'] == second_planet and x['c_aspect'] == aspect) or \
                                           (x['n_planet'] == first_planet and x['c_planet'] == second_planet and x['c_aspect'] == aspect) ]
    return 1 if len(found_params) > 0 else 0

def is_retrograde(row, planet):
    if planet in [SUN,MOON]:
        return 0
    c_chart = charts[row['CorrectedDate']]
    return 1 if c_chart['planets']['planets_retrograde'][planet] else 0   

def create_booster_swing_trade(eta,depth,num_trees, train_x, train_y, test_x, test_y, columns, trained_model):
    param['max_depth'] = depth
    param['eta'] = eta
    num_round = num_trees
    dtrain = xgb.DMatrix(train_x, train_y, feature_names = columns)
    dtest = xgb.DMatrix(test_x, test_y, feature_names = columns)
    train_labels = dtrain.get_label()
    ratio = float(np.sum(train_labels == 0)) / np.sum(train_labels == 1) 
    param['scale_pos_weight'] = ratio
    gpu_res = {}
    booster = xgb.train(param, dtrain, num_round, evals_result=gpu_res, evals = [], xgb_model = trained_model)    
    return booster

def get_best_booster(target_variable, max_interactions):
    booster = None
    best_score = 1
    best_booster = None
    for current_run in range(max_interactions):
        X = StockPrices[astro_columns].values
        Y = StockPrices[target_variable].values
        total_test = xgb.DMatrix(X, feature_names = astro_columns)
        X_train_1, X_train_2, y_train_1, y_train_2 = ttsplit(X, Y, 
                                                             test_size=0.3, 
                                                             random_state=None,
                                                             shuffle=True)
        booster = create_booster_swing_trade(
            ETA, DEPTH, NUM_TREES, 
            X_train_1, y_train_1, 
            X_train_2, y_train_2, 
            astro_columns, booster)
        current_score = mse(booster.predict(total_test), Y)
        if current_score < best_score:
            best_score = current_score
            best_booster = booster
    return best_booster, best_score

def predict_score(row, booster, df):
    matrix_to_predict = row[astro_columns].values
    matrix_to_predict = matrix_to_predict.reshape((1,-1))
    row_features = xgb.DMatrix(matrix_to_predict, feature_names = astro_columns)
    predicted = booster.predict(row_features)[0]
    if predicted > 0.95:
        #corrected_row = df[df['CorrectedDate'] == row['CorrectedDate']][astro_columns]
        #display(eli5.explain_prediction(booster, doc=corrected_row, feature_names  = astro_columns))
        pass
    return predicted

In [ ]:
max_counter = StockPrices['Counter'].max()

StockPrices['CorrectedDate'] = StockPrices.apply( lambda x :  correct_date(x), axis =1 )
StockPrices['PreviousStartPrice'] = StockPrices.apply( lambda x :  get_previous_stock_price(StockPrices, x, SWING_TRADE_DURATION), axis =1 )
StockPrices['FutureFinalPrice'] = StockPrices.apply( lambda x :  get_future_stock_price(StockPrices, x, max_counter, SWING_TRADE_DURATION), axis =1 )
StockPrices['PreviousStartDate'] = StockPrices.apply( lambda x :  get_previous_stock_date(StockPrices, x, SWING_TRADE_DURATION), axis =1 )
StockPrices['FutureFinalDate'] = StockPrices.apply( lambda x :  get_future_stock_date(StockPrices, x, max_counter, SWING_TRADE_DURATION), axis =1 )

StockPrices['CurrentTrend'] = StockPrices.apply(lambda x : calculate_current_trend(x), axis = 1)

StockPrices['FutureTrend'] = StockPrices.apply(lambda x : calculate_future_trend(x), axis = 1)
StockPrices['SwingStrength'] = StockPrices.apply(lambda x: calculate_swing_strenght(x), axis =1)
StockPrices['IntradayVolatility'] = StockPrices.apply(lambda x: calculate_intraday_volatility(StockPrices, x, SWING_TRADE_DURATION), axis =1)

StockPrices['FutureTrendMax'] = StockPrices.apply(lambda x : get_future_stock_max_price(StockPrices, x, max_counter, SWING_TRADE_DURATION), axis = 1)
StockPrices['FutureTrendMin'] = StockPrices.apply(lambda x : get_future_stock_min_price(StockPrices, x, max_counter, SWING_TRADE_DURATION), axis = 1)

StockPrices['IsSwing'] = StockPrices.apply(lambda x: detect_swing_trade(x, SWING_EXPECTED_VOLATILITY), axis =1)
StockPrices['IsSwing'] = StockPrices.apply(lambda x:clean_swing_trade(StockPrices, x, SWING_EXPECTED_VOLATILITY), axis =1)

StockPrices['StockIncreasedPrice'] = StockPrices.apply(lambda x:detect_price_increase(x, STAGNATION_THRESHOLD), axis =1)

StockPrices['StockDecreasedPrice'] = StockPrices.apply(lambda x:detect_price_decrease(x, STAGNATION_THRESHOLD), axis =1)

StockPrices['StockStagnated'] = StockPrices.apply(lambda x:detect_price_stagnated(x, STAGNATION_THRESHOLD), axis =1)

StockPrices['TARGET_IS_SWING'] = StockPrices['IsSwing']
StockPrices['TARGET_PRICE_INCREASE'] = StockPrices['StockIncreasedPrice']
StockPrices['TARGET_PRICE_DECREASE'] = StockPrices['StockDecreasedPrice']
StockPrices['TARGET_PRICE_STAGNATION'] = StockPrices['StockStagnated']

In [ ]:
output_excel_file='./output/{}.Analisys.xlsx'.format(ASSET_TO_CALCULATE)
StockPrices.to_excel(output_excel_file)

In [ ]:
#StockPrices[StockPrices['IsSwing'] == 1].sort_values(by=['Date'], ascending=[0]).head(20)

In [ ]:
#StockPrices[StockPrices['StockIncreasedPrice'] == 1].sort_values(by=['Date'], ascending=[0]).head(20)

In [ ]:
#StockPrices[StockPrices['StockDecreasedPrice'] == 1].sort_values(by=['Date'], ascending=[0]).head(20)

In [ ]:
swing_to_chart = []
for index, current_swing in StockPrices[StockPrices['IsSwing'] == 1].iterrows():
    swing_to_chart.append(dict(
        x0=current_swing['CorrectedDate'], 
        x1=current_swing['CorrectedDate'], 
        y0=0, 
        y1=1, 
        xref='x', 
        yref='paper',
        line_width=2))


In [ ]:
fig = go.Figure(data=[go.Candlestick(
                x=StockPrices['CorrectedDate'],
                open=StockPrices['Open'],
                high=StockPrices['High'],
                low=StockPrices['Low'],
                close=StockPrices['Price'])])
fig.update_layout(
    title="{} Detected Swing Trade Opportunities".format(ASSET_TO_CALCULATE),
    width=1000,
    height=500,
    xaxis_rangeslider_visible=False,
    shapes=swing_to_chart,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=30,
        pad=4
    ),    
)
fig.show()

In [ ]:
asset_natal_chart_input = create_input_json(NATAL_DATE, 
                                            DEFAULT_PARAMETERS, 
                                            DEFAULT_CONFIG)

asset_natal_chart = calculate_chart(asset_natal_chart_input)
dates_to_generate = list(StockPrices['CorrectedDate'])

In [ ]:
charts = {}
transits = {}

for current_date in dates_to_generate:
    chart_input = create_input_json(current_date + 'T10:00:00-03:00', 
                                      DEFAULT_PARAMETERS, 
                                      DEFAULT_CONFIG)
    charts[current_date] = calculate_chart(chart_input)
    transits[current_date] = calculate_transits(asset_natal_chart, charts[current_date], PLANETS_TO_CALCULATE, ASPECTS_TO_CALCULATE, 4)    

In [ ]:
astro_columns = []

for first_planet in PLANETS_TO_CALCULATE:
    for second_planet in PLANETS_TO_CALCULATE:
        for aspect in ASPECTS_TO_CALCULATE:
            column_name="ASTRO_{}_{}_{}".format(PLANETS[first_planet],ASPECT_NAME[aspect],PLANETS[second_planet]).upper()
            astro_columns.append(column_name)
            StockPrices['StockIncreasedPrice'] = StockPrices.apply(lambda x:detect_price_increase(x, STAGNATION_THRESHOLD), axis =1)
            StockPrices[column_name] = StockPrices.apply(lambda x:is_aspected(x, first_planet, second_planet, aspect), axis =1)
            StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float', errors='coerce')

In [ ]:
for first_planet in PLANETS_TO_CALCULATE:
    column_name="ASTRO_{}_RETROGADE".format(PLANETS[first_planet]).upper()
    astro_columns.append(column_name)
    StockPrices[column_name] = StockPrices.apply(lambda x:is_retrograde(x, first_planet), axis =1)
    StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float',errors='coerce')

In [ ]:
booster_swing_trade, score_swing_trade = get_best_booster('TARGET_IS_SWING', MAX_INTERACTIONS)
print("Best Score for Swing Trade Model:{}".format(score_swing_trade))

booster_price_increase, score_price_increase = get_best_booster('TARGET_PRICE_INCREASE', MAX_INTERACTIONS)
print("Best Score for Price Increase Model:{}".format(score_price_increase))

booster_price_decrease, score_price_decrease = get_best_booster('TARGET_PRICE_DECREASE', MAX_INTERACTIONS)
print("Best Score for Price Decrease Model:{}".format(score_price_decrease))

booster_price_stagnation, score_price_stagnation = get_best_booster('TARGET_PRICE_STAGNATION', MAX_INTERACTIONS)
print("Best Score for Price Stagnation Model:{}".format(score_price_stagnation))

relevant_features_swing_trade = sorted( ((v,k) for k,v in booster_swing_trade.get_score().items()), reverse=True)
relevant_features_price_increase = sorted( ((v,k) for k,v in booster_price_increase.get_score().items()), reverse=True)
relevant_features_price_decrease = sorted( ((v,k) for k,v in booster_price_decrease.get_score().items()), reverse=True)
relevant_features_price_stagnation = sorted( ((v,k) for k,v in booster_price_stagnation.get_score().items()), reverse=True)

In [ ]:
StockPrices['PredictSwingTradeScore'] = StockPrices.apply(lambda x:predict_score(x, booster_swing_trade, StockPrices), axis =1)
StockPrices['PredictPriceIncreaseScore'] = StockPrices.apply(lambda x:predict_score(x, booster_price_increase, StockPrices), axis =1)
StockPrices['PredictPriceDecreaseScore'] = StockPrices.apply(lambda x:predict_score(x, booster_price_decrease, StockPrices), axis =1)
StockPrices['PredictPriceStagnation'] = StockPrices.apply(lambda x:predict_score(x, booster_price_stagnation, StockPrices), axis =1)

In [ ]:
booster_swing_trade.save_model('./output/{}_swing_trade.model'.format(ASSET_TO_CALCULATE))
booster_price_increase.save_model('./output/{}_price_increase.model'.format(ASSET_TO_CALCULATE))
booster_price_decrease.save_model('./output/{}_price_decrease.model'.format(ASSET_TO_CALCULATE))
booster_price_stagnation.save_model('./output/{}_price_stagnation.model'.format(ASSET_TO_CALCULATE))

In [ ]:
swing_to_chart = []
for index, current_swing in StockPrices[StockPrices['PredictSwingTradeScore'] > 0.9].iterrows():
    swing_to_chart.append(dict(
        x0=current_swing['CorrectedDate'], 
        x1=current_swing['CorrectedDate'], 
        y0=0, 
        y1=1, 
        xref='x', 
        yref='paper',
        line_width=2))

In [ ]:
fig = go.Figure(data=[go.Candlestick(
                x=StockPrices['CorrectedDate'],
                open=StockPrices['Open'],
                high=StockPrices['High'],
                low=StockPrices['Low'],
                close=StockPrices['Price'])])
fig.update_layout(
    title="{} Swing Trade Opportunities detected by XGBoost".format(ASSET_TO_CALCULATE),
    width=1000,
    height=500,
    xaxis_rangeslider_visible=False,
    shapes=swing_to_chart,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=30,
        pad=4
    ),    
)
fig.show()

In [ ]:
swing_to_chart = []
for index, current_swing in StockPrices[StockPrices['PredictPriceIncreaseScore'] > 0.9].iterrows():
    swing_to_chart.append(dict(
        x0=current_swing['CorrectedDate'], 
        x1=current_swing['CorrectedDate'], 
        y0=0, 
        y1=1, 
        xref='x', 
        yref='paper',
        line_width=2))

In [ ]:
fig = go.Figure(data=[go.Candlestick(
                x=StockPrices['CorrectedDate'],
                open=StockPrices['Open'],
                high=StockPrices['High'],
                low=StockPrices['Low'],
                close=StockPrices['Price'])])
fig.update_layout(
    title="{} Price Increase Opportunities detected by XGBoost (Min {}%)".format(ASSET_TO_CALCULATE, STAGNATION_THRESHOLD),
    width=1000,
    height=500,
    xaxis_rangeslider_visible=False,
    shapes=swing_to_chart,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=30,
        pad=4
    ),    
)
fig.show()

In [ ]:
output_excel_file='./output/{}.Analisys.xlsx'.format(ASSET_TO_CALCULATE)
StockPrices.to_excel(output_excel_file)